In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
def load_data(filepath):
    h5f = h5py.File(filepath, 'r')
    X_train = h5f['X_train'][:]
    y_train = h5f['y_train'][:]
    X_test = h5f['X_test'][:]
    y_test = h5f['y_test'][:]
    h5f.close()
    return (X_train, y_train), (X_test, y_test)

(X_train, y_train), (X_test, y_test) = load_data('data/RockAI_test_50_images.h5')

num_classes = 2

#Pre-process the data
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [5]:
import numpy as np

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
# datagen = ImageDataGenerator()
datagen.fit(X_train)

In [14]:
image_size = (224, 224)

model = tf.keras.models.Sequential()
model.add(VGG16(weights='imagenet'))
# model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

In [15]:
batch_size = 32
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

historytemp = model.fit(datagen.flow(X_train, y_train,
                        batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=10)

Epoch 1/10
39/39 [==============================] - 8s 193ms/step - loss: 0.6305 - accuracy: 0.6503
Epoch 2/10
39/39 [==============================] - 8s 194ms/step - loss: 0.2845 - accuracy: 0.9416
Epoch 3/10
39/39 [==============================] - 8s 194ms/step - loss: 0.2116 - accuracy: 0.9470
Epoch 4/10
39/39 [==============================] - 8s 195ms/step - loss: 0.1899 - accuracy: 0.9549
Epoch 5/10
39/39 [==============================] - 8s 195ms/step - loss: 0.2368 - accuracy: 0.9397
Epoch 6/10
39/39 [==============================] - 8s 195ms/step - loss: 0.2216 - accuracy: 0.9443
Epoch 7/10
39/39 [==============================] - 8s 195ms/step - loss: 0.1928 - accuracy: 0.9545
Epoch 8/10
39/39 [==============================] - 8s 195ms/step - loss: 0.2159 - accuracy: 0.9452
Epoch 9/10
39/39 [==============================] - 8s 198ms/step - loss: 0.2117 - accuracy: 0.9463
Epoch 10/10
39/39 [==============================] - 8s 196ms/step - loss: 0.2261 - accuracy: 0.9410

In [16]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 51ms/step - loss: 1.5165 - accuracy: 0.5000


[1.5165082216262817, 0.5]

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='sgd', metrics=[tf.keras.metrics.AUC()])

historytemp = model.fit(datagen.flow(X_train, y_train,
                        batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=15)

In [ ]:
model.evaluate(X_test, y_test)